In [1]:
!apt install swig cmake
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit1/requirements-unit1.txt

!sudo apt-get update
!sudo apt-get install -y python3-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 45 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 1s (1,090 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubu

In [ ]:
import os
os.kill(os.getpid(),9)

In [9]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy

# Create environment
env = gym.make("LunarLander-v2", render_mode="rgb_array")

# Instantiate the agent
model = PPO(
    policy='MlpPolicy',
    env=env,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    gamma=0.999999999,
    gae_lambda=0.95,
    ent_coef=0.01,
    verbose=1
)

# Train the agent and display a progress bar
model.learn(total_timesteps=int(500000), progress_bar=True)

# Save the agent
model.save("ppo_lunar3")
#del model  # delete trained model to demonstrate loading

Output()

Streaming output truncated to the last 5000 lines.
|    value_loss           | 989         |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 113          |
|    ep_rew_mean          | -161         |
| time/                   |              |
|    fps                  | 383          |
|    iterations           | 7            |
|    time_elapsed         | 37           |
|    total_timesteps      | 14336        |
| train/                  |              |
|    approx_kl            | 0.0047298213 |
|    clip_fraction        | 0.0531       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.31        |
|    explained_variance   | -0.00456     |
|    learning_rate        | 0.0003       |
|    loss                 | 367          |
|    n_updates            | 60           |
|    policy_gradient_loss | -0.00549     |
|    value_loss           | 1.13e+03     |
-----

In [10]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
import imageio
import numpy as np

# Load the trained agent
env = gym.make("LunarLander-v2", render_mode="rgb_array")
model = PPO.load("ppo_lunar3", env=env)

# Evaluate the agent
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)
print('Mean reward:', mean_reward, 'Std. reward:', std_reward)

# Test the trained agent
images = []
j = 0
vec_env = model.get_env()
obs = vec_env.reset()
img = model.env.render(mode="rgb_array")
for i in range(2000):
    images.append(img)
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = vec_env.step(action)
    #vec_env.render()
    if dones:
        j = j + 1
        print('Episode:', j, 'Rewards:', rewards, 'Info:', info, 'Dones:', dones)
        imageio.mimsave(f"ppo_lander2_{j}.gif", [np.array(img) for k, img in enumerate(images) if k%2 == 0], fps=29)
        images = []
    img = model.env.render(mode="rgb_array")
vec_env.close()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Mean reward: 268.0203894 Std. reward: 20.22592088132768
Episode: 1 Rewards: [100.] Info: [{'episode': {'r': 274.338271, 'l': 300, 't': 5.665108}, 'TimeLimit.truncated': False, 'terminal_observation': array([ 5.5519961e-02, -7.8366039e-04,  0.0000000e+00,  0.0000000e+00,
        2.9740625e-03,  0.0000000e+00,  1.0000000e+00,  1.0000000e+00],
      dtype=float32)}] Dones: [ True]
Episode: 2 Rewards: [100.] Info: [{'episode': {'r': 244.740246, 'l': 313, 't': 12.263534}, 'TimeLimit.truncated': False, 'terminal_observation': array([ 9.5917704e-03, -9.1294287e-04,  0.0000000e+00,  0.0000000e+00,
        1.8391449e-03,  0.0000000e+00,  1.0000000e+00,  1.0000000e+00],
      dtype=float32)}] Dones: [ True]
Episode: 3 Rewards: [100.] Info: [{'episode': {'r': 259.166814, 'l': 322, 't': 18.911897}, 'TimeLimit.truncated': False, 'terminal_observation': array([-2.5348186e-02, -8.1209181e-04,  0.0000000e+00,  0.0000000e+00,
